# INTEGER PROGRAMMING AND DECISION ANALYSIS

**1) To graduate from Basketweavers University with a major in operations research, a 
student must complete at least two math courses, at least two OR courses, and at least 
two computer courses. Some courses can be used to fulfill more than one requirement: 
Calculus can fulfill the math requirement; operations research, math and OR 
requirements; data structures, computer and math requirements; business statistics, 
math and OR requirements; computer simulation, OR and computer requirements; 
introduction to computer programming, computer requirement; and forecasting, OR 
and math requirements. Some courses are prerequisites for others: Calculus is a 
prerequisite for business statistics; introduction to computer programming is a 
prerequisite for computer simulation and for data structures; and business statistics is 
a prerequisite for forecasting. Formulate an IP that minimizes the number of courses 
needed to satisfy the major requirements demands.Clearly define decision variables, 
objective function and constraints. Solve the problem using PULP and report the code 
and output.**

Decision variables:

Decision variables (x1,2,3,4,5,6) are binary; 

taking related course is 1,

no taking related course is 0.

x1 = taking calculus course (0-1)

x2 = taking operations research course (0-1)

x3 = taking data structures course (0-1)

x4 = taking business statistics course (0-1)

x5 = taking computer simulation course (0-1)

x6 = taking introduction to computer programming course (0-1)

x7 = taking forecasting course (0-1)

Objective function (minimize to taking number of courses):

Min z = x1 + x2 + x3 + x4 + x5 + x6 + x7

Constraints subject to:

For fullfilling math courses' requirements:

x1 + x2 + x3 + x4 + x7 >= 2

For fullfilling OR courses' requirements:

x2 + x4 + x5 + x7 >= 2

For fullfilling computer courses' requirements:

x3 + x5 + x6 >= 2

Calculus is prerequisite of business statistics:

x1 >= x4

Introduction to computer programming is prerequisite of computer simulation and data structures:

x6 >= x5

x6 >= x3

Business statistics is prerequisite of forecasting:

x4 >= x7

Variables are binary:

x1,x2,x3,x4,x5,x6,x7 ∈ (0,1)

In [16]:
#Problem is created

from pulp import *

prob = LpProblem("Min_Taking_Courses", LpMinimize)

In [17]:
#Decision variables are added to the problem

x1 = LpVariable("Calculus", 0, cat = LpBinary)
x2 = LpVariable("OperationsRes.", 0, cat = LpBinary)
x3 = LpVariable("DataStructures", 0,  cat = LpBinary)
x4 = LpVariable("BusinessStat", 0, cat = LpBinary)
x5 = LpVariable("Comp.Sim.", 0, cat = LpBinary)
x6 = LpVariable("Intro.Com.", 0,  cat = LpBinary)
x7 = LpVariable("Forecasting", 0,  cat = LpBinary)

In [18]:
#Objective function is assigned

prob += x1+x2+x3+x4+x5+x6+x7

In [19]:
#Constraints are added

prob += x1+x2+x3+x4+x7 >= 2, "Math Requirement"
prob += x2+x4+x5+x7 >= 2, "OR Requirement"
prob += x3+x5+x6 >= 2, "Computer Requirement"
prob += x1 >= x4, "Prereq. Bussiness Stat."
prob += x6 >= x5, "Prereq. Comp. Sim."
prob += x6 >= x3, "Prereq. Data Str."
prob += x4 >= x7, "Prereq. Forecasting"

In [20]:
prob.solve()

1

In [21]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [22]:
#The problem is solved, and it gives optimal solution. Now, for each decision variables, optimum values are printed:

for i in prob.variables():
    print(i.name, "=", i.varValue)

BusinessStat = 0.0
Calculus = 0.0
Comp.Sim. = 1.0
DataStructures = 1.0
Forecasting = 0.0
Intro.Com. = 1.0
OperationsRes. = 1.0


In [23]:
#Then, optimum value of the objective function is:

print("Minimum Courses which must taken = ", value(prob.objective))

Minimum Courses which must taken =  4.0


To successfully graduate with a major in operations research from Basketweavers University, students are required to take at least these four courses: computer simulation, data structures, introduction to computer programming, and operations research.

**2) Glueco produces three types of glue on two different production lines. Each line can be 
utilized by up to seven workers at a time. Workers are paid 500 dollars per week on 
production line 1, and 900 dollars per week on production line 2. A week of production costs 
1,000 dollars to set up production line 1 and 2,000 dollars to set up production line 2. During a week 
on a production line, each worker produces the number of units of glue shown in the 
following Table.**

![title](ab.png)

**Each week, at least 120 units of glue 1, at least 150 units of glue 2, and at least 200 
units of glue 3 must be produced. Formulate an IP to minimize the total cost of 
meeting weekly demands. Clearly define decision variables, objective function and 
constraints. Solve the problem using PULP and report your code and output.**


Decision variables:
    
x1 = number of labor for line 1

x2 = number of labor for line 2

y1 = line 1 was used or not (binary, 0-1)

y2 = line 2 was used or not (binary, 0-1)

Objective function (minimize the total cost):

Min z = 1000 * y1 + 2000 * y2 + 500 * x1 + 900 * x2

Constraints subject to:

Weekly demand for glue 1:

20 * x1 + 50 * x2 >= 120

Weekly demand for glue 2:

30 * x1 + 35 * x2 >= 150

Weekly demand for glue 3:

40 * x1 + 45 * x2 >= 200

Each line contains maximum 7 workers:

x1 <= 7 * y1

x2 <= 7 * y2

Non-negativity and binary constraints:

x1, x2 >= 0

y1, y2 ∈ (0,1)

In [24]:
#Problem:

prob2 = LpProblem("Min_Total_Costs", LpMinimize)

In [25]:
#Decision variables:

x1 = LpVariable("Line1_workers", 0, cat = LpInteger)
x2 = LpVariable("Line2_workers", 0, cat = LpInteger)
y1 = LpVariable("Usage_Line1", 0, cat = LpBinary)
y2 = LpVariable("Usage_Line2", 0, cat = LpBinary)

In [26]:
#Objective function:

prob2 += 1000*y1+2000*y2+500*x1+900*x2

In [27]:
#Constraints:

prob2 += 20*x1+50*x2 >= 120, "Unit of production Glue 1"
prob2 += 30*x1+35*x2 >= 150, "Unit of production Glue 2"
prob2 += 40*x1+45*x2 >= 200, "Unit of production Glue 3"
prob2 += x1 <= 7*y1, "Workers in line 1"
prob2 += x2 <= 7*y2, "Workers in line 2"

In [28]:
prob2.solve()
print("Status:", LpStatus[prob2.status])

Status: Optimal


In [29]:
for i in prob2.variables():
    print(i.name, "=", i.varValue)

Line1_workers = 6.0
Line2_workers = 0.0
Usage_Line1 = 1.0
Usage_Line2 = 0.0


In [30]:
print("Minimum Cost = ", value(prob2.objective))

Minimum Cost =  4000.0


The findings indicate that by exclusively utilizing Line 1 with a workforce of 6 employees, Glueco can fulfill the weekly demand for glue types 1, 2, and 3, while minimizing the associated costs to 4,000 dollars.

**3) A monopolist can purchase up to 17.25 oz of a chemical for 10 dollars/oz. At a cost of 3 dollars/oz, 
the chemical can be processed into an ounce of product 1; or, at a cost of 5 dollars/oz, the 
chemical can be processed into an ounce of product 2. If x1 oz of product 1 are 
produced, it sells for a price of 30-x1 per ounce. If x2 oz of product 2 are produced, it 
sells for a price of 50-2x2 per ounce. Determine how the monopolist can maximize 
profits. (Hint: Be aware that you need to transform the problem into a minimization 
problem and flip the direction of ≤ constraints by multiplying with -1 to solve the problem 
in scipy)**

Decision variables:

x1 = produced unit of product 1

x2 = produced unit of product 2

Objective function (maximize the profit):

Max z = x1 * (30 - x1) + x2 * (50 - 2 * x2) - (10 + 3 * x1 + 5 * x2)

This function is changed to minimization as:

Min z = - x1 * (30 - x1) - x2 * (50 - 2 * x2) + (10 + 3 * x1 + 5 * x2)

Constraints subject to:

There are 17.25 oz of chemical:

x1 + x2 <= 17.25

Non-negativity constraints:

x1, x2 >= 0

In [31]:
from scipy.optimize import minimize

#Objective function is defined as minimization

def objective_function(x):
    return -(30 - x[0]) * x[0] - (50 - 2 * x[1]) * x[1] + (10 + 3 * x[0] + 5 * x[1])

In [32]:
#Constraints are defined

constraints = ({'type': 'ineq', 'fun': lambda x: 17.25 - (x[0] + x[1])},
               {'type': 'ineq', 'fun': lambda x: x[0]},
               {'type': 'ineq', 'fun': lambda x: x[1]})

In [33]:
#Starting point is created

starting = [0, 0]

#Minimization solver is applied

result = minimize(objective_function, starting, constraints=constraints)

In [34]:
#Optimal solution is printed

optimal_solution = result.x

print("Optimal Solution:")
print("x1 =", optimal_solution[0])
print("x2 =", optimal_solution[1])
print("Maximum Profit =", -result.fun)  #The negative of the objective function value gives the maximum profit

Optimal Solution:
x1 = 8.49999936421625
x2 = 8.750000635782825
Maximum Profit = 387.87499999998954


Results showed that the monopolist can reach maximum 387.499 dollars with producing 8.49 oz of product 1, and 8.75 oz of product 2.

**4) During the summer, Olympic swimmer Adam Johnson swims every day. On sunny 
summer days, he goes to an outdoor pool, where he may swim for no charge. On rainy
days, he must go to a domed pool. At the beginning of the summer, he has the option 
of purchasing a 15 dollars season pass to the domed pool, which allows him use for the entire 
summer. If he doesn’t buy the season pass, he must pay 1 dollar each time he goes there. 
Past meteorological records indicate that there is a 60 percentage chance that the summer will 
be sunny (in which case there is an average of 6 rainy days during the summer) and a 
40 percentage chance the summer will be rainy (an average of 30 rainy days during the summer). 
Before the summer begins, Adam has the option of purchasing a long-range weather 
forecast for 1 dollar. The forecast predicts a sunny summer 80 percentage of the time and a rainy 
summer 20 percentage of the time. If the forecast predicts a sunny summer, there is a 70 percentage 
chance that the summer will actually be sunny. If the forecast predicts a rainy summer, 
there is an 80 percentage chance that the summer will actually be rainy. Assuming that Adam’s 
goal is to minimize his total expected cost for the summer, what should he do? Also 
find EVSI and EVPI**

![EVSI](soru4-1.png)

There are two possible outcomes for the weather condition in the summer. The probability of it being rainy is 0.4, while the probability of it being sunny is 0.6. If the summer turns out to be sunny, Adam has two options: he can either purchase a season pass or not. If he chooses to buy a season pass, it would cost him 15 dollars. If he decides against purchasing the season pass, he will need to pay 6 dollars for entering the domed pool on the 6 rainy days in the sunny summer. Comparing these options for a sunny summer, Adam pays less if he opts not to purchase the season pass because it costs him only 6 dollars. On the other hand, if the summer turns out to be rainy, he again has the choice of buying the season pass for 15 dollars or not buying it. If he doesn't buy the season pass, he would have to pay 30 dollars for entering the domed pool on the 30 rainy days. Considering these options, buying the season pass (15 dollars) is less expensive than the alternative. Overall, by multiplying the relevant probabilities with 6 dollars and 15 dollars and summarizing them (6 * 0.6 + 15 * 0.4 = 9.6 dollars), 9.6 dollars represents the expected value of perfect information. If Adam were to purchase the season pass directly, it would cost him 15 dollars. Calculating the difference between 9.6 dollars and 15 dollars (-9.6-(-15) = 5.4 dollars), it is found that 5.4 dollars is the upper limit for the expected value of perfect information (EVPI). This means that Adam can pay a maximum of 5.4 dollars for absolute information about whether the summer will be rainy or sunny.

![soru](soru4-2.png)

At the beginning, Adam has two options: purchasing the long-range weather forecast or not. As shown above, 

**if Adam does not purchase the forecast,** he again has two options: buying a season pass or not. If Adam buys the season pass, he pays 15 dollars. However, if he chooses not to buy the season pass, there are two possible events with a 0.6 probability of a sunny summer and a 0.4 probability of a rainy summer. If the summer turns out to be sunny, with an average of 6 rainy days, Adam pays 1 dollar for entering the domed pool for these six days, totaling 6 dollars. On the other hand, if the summer is rainy, with an average of 30 rainy days, he pays 30 dollars. Multiplying these costs by the probability values: 6 * 0.6 + 30 * 0.4 = 15.6 dollars. As seen in the decision tree, a cost of 15 dollars is a better choice than a cost of 15.6 dollars. Thus, if Adam does not purchase the long-range weather forecast, his minimum cost is 15 dollars.

**If Adam purchases the long-range forecast,** there would be two possible events: a sunny summer prediction with a probability of 0.8 and a rainy summer prediction with a probability of 0.2.

**If a sunny summer is predicted,** Adam faces two decisions: buying a season pass or not. If he buys the season pass, he will pay 16 dollars (15 dollars for the season pass, 1 dollar for the forecast). However, if he decides not to buy the season pass, there will be two possible outcomes: a sunny summer with a probability of 0.7 and a rainy summer with a probability of 0.3. If the summer turns out to be sunny, with an average of six rainy days, he pays 7 dollars (6 * 1 dollar for entering the domed pool, 1 dollar for the forecast). However, if the summer is rainy, with an average of 30 rainy days, Adam pays 31 dollars (30 * 1 dollar for entering the domed pool, 1 dollar for the forecast). If these values are multiplied by their respective probabilities 7 * 0.7 + 31 * 0.3 = 14.2 dollars, Adam pays a minimum of 14.2 dollars. When comparing 14.2 dollars and 16 dollars, a cost of 14.2 dollars is better than 16 dollars.

**If a rainy summer is predicted by the forecast,** Adam again has two options. He can buy the season pass for 16 dollars, or he can choose not to buy the season pass. If he decides not to buy the season pass, there would be two possible outcomes: a sunny summer with a probability of 0.2 or a rainy summer with a probability of 0.8. If the summer turns out to be sunny, he pays only 7 dollars. On the other hand, if the summer is rainy, he pays 31 dollars. Again, these values are multiplied by their respective probability values 31 * 0.8 + 7 * 0.2 = 26.2 dollars, so Adam may pay 26.2 dollars if he does not buy the season pass while the long-range forecast predicts a rainy summer. When comparing 26.2 dollars and 16 dollars, paying 16 dollars is a better choice than paying 26.2 dollars.

When multiplying the related probabilities with 16 dollars (predicted rainy summer) and 14.2 dollars (predicted sunny summer) and summing them: 14.2 * 0.8 + 16 * 0.2 = 14.2 dollars, Adam pays a minimum of 14.2 dollars. If 14.2 dollars are compared with 15 dollars, paying 14.2 dollars would be a better choice for Adam. Therefore, if Adam purchases the long-range weather forecast, he pays less than the no-purchasing choice.

When comparing the first decision tree, Adam pays 1 dollar more for the long-range weather forecast in addition to the season pass. Also, if he does not purchase this forecast, he buys the season pass for 15 dollars rather than 16 dollars. Thus, the difference between these values (16 - 15 = 1 dollar) gives the value of sample information (EVSI). In this problem, EVSI is 1 dollar.

**The optimum decision for Adam** would be purchasing long-range weather forecast information, and if it predicts a sunny summer, Adam should not purchase the season pass. Besides that, if the forecast predicts a rainy summer, Adam should purchase the season pass.